In [1]:
import pandas as pd
import os

os.chdir("C:/Users/jstelman/Git/chelsvig_urban_pesticides/src/")
from path_names import *

obs_path = os.path.join(main_path, "observed_data", "SURF_water_placer_bifenthrin.csv")
print(obs_path)

outfalls = ['outfall_31_26', 'outfall_31_28', 'outfall_31_29', 'outfall_31_35', 
            'outfall_31_36', 'outfall_31_38', 'outfall_31_42']

def output_path(of, i):
    op_path = os.path.join(vvwm_path, of, ("input_" + str(i)), "output_NPlesant_Custom_parent_daily.csv")
    return(op_path)

main_path   C:\Users\jstelman\Git\chelsvig_urban_pesticides
dir_path    C:\Users\jstelman\Git\chelsvig_urban_pesticides\probabilistic_python
exe_path    C:\Users\jstelman\Git\chelsvig_urban_pesticides\probabilistic_python\exe
swmm_path   C:\Users\jstelman\Git\chelsvig_urban_pesticides\probabilistic_python\input\swmm
inp_path    C:\Users\jstelman\Git\chelsvig_urban_pesticides\probabilistic_python\input\swmm\NPlesantCreek.inp
bin_path    C:\Users\jstelman\Git\chelsvig_urban_pesticides\probabilistic_python\input\swmm\NPlesantCreek.out
vvwm_path   C:\Users\jstelman\Git\chelsvig_urban_pesticides\probabilistic_python\input\vvwm
wet_path    C:\Users\jstelman\Git\chelsvig_urban_pesticides\probabilistic_python\weather
C:\Users\jstelman\Git\chelsvig_urban_pesticides\observed_data\SURF_water_placer_bifenthrin.csv


In [2]:
# Import Observed Data
obs_data = pd.read_csv(obs_path, usecols=["Concentration..ppb.","Level_of_quantification..ppb.",
                                         "Method_detection_level..ppb.", "Sample_date", "Site_code"],
                      parse_dates=["Sample_date"])

# do Proxy
for idx, row in obs_data[obs_data['Concentration..ppb.']==0].iterrows():
    obs_data.loc[idx,'Concentration..ppb.'] = min([row['Level_of_quantification..ppb.'],
                                                   row['Method_detection_level..ppb.']])/2

# drop columns
obs_data = obs_data.drop(labels = ["Level_of_quantification..ppb.","Method_detection_level..ppb."], axis = 1)
obs_data.head()

,Concentration..ppb.,Sample_date,Site_code
0,0.00235,2015-06-08,31_26
1,0.00235,2010-06-24,31_44
2,0.00235,2009-05-06,31_44
3,0.00235,2010-09-03,31_44
4,0.00035,2016-10-14,31_26


## For 1 Simulation

In [52]:
# For first outfall:

# read in simulated data and convert so units match units of observed data
sim1_of26_data = pd.read_csv(output_path("outfall_31_26", 1), usecols = [1], skiprows=5, names = ["davg_bif_conc"])*1000000
# add date and Site_code columns with same headers as in observed data frame
sim1_of26_data['Sample_date'] = pd.date_range(start='1/1/2009', periods=3287, freq='D')
sim1_of26_data['Site_code'] = '31_26'
sim1_of26_data.head()

,davg_bif_conc,Sample_date,Site_code
0,0.0,2009-01-01,31_26
1,0.0,2009-01-02,31_26
2,0.0,2009-01-03,31_26
3,0.0,2009-01-04,31_26
4,0.0,2009-01-05,31_26


In [53]:
# merge the observed data with the simulated data
comp_data_ip1 = sim1_of26_data.merge(obs_data, how = "inner", on = ['Sample_date','Site_code'])

In [54]:
# now repeat for rest of outfalls, appending to the data frame as we go
for of in outfalls[1:]:
    dat = pd.read_csv(output_path(of, 1), usecols = [1], skiprows=5, names = ["davg_bif_conc"])*1000000
    dat['Sample_date'] = pd.date_range(start='1/1/2009', periods=3287, freq='D')
    dat['Site_code'] = of[-5:]
    comp_data_ip1 = comp_data_ip1.append(dat.merge(obs_data, how = "inner", on = ['Sample_date','Site_code']), ignore_index=True)

In [55]:
# not too shabby
comp_data_ip1

,davg_bif_conc,Sample_date,Site_code,Concentration..ppb.
0,1.744700e+07,2009-02-13,31_26,0.017000
1,8.595200e+07,2009-04-08,31_26,0.007740
2,7.793400e+09,2009-04-13,31_26,0.000880
3,1.738700e+11,2009-05-01,31_26,0.008800
4,1.026300e+10,2009-08-28,31_26,0.000880
...,...,...,...,...
101,1.385500e+10,2010-07-19,31_42,0.000880
102,5.664500e+07,2010-10-24,31_42,0.000880
103,7.886300e+10,2011-02-16,31_42,0.000880
104,6.527900e+10,2011-05-15,31_42,0.000455


In [68]:
# Calc Nash-Sutcliffe Efficiency for Simulation 1
NSE1 = 1 - ((sum(comp_data_ip1['davg_bif_conc'] - comp_data_ip1['Concentration..ppb.'])**2)/
            (sum(comp_data_ip1['Concentration..ppb.'] - 
                 sum(comp_data_ip1['Concentration..ppb.'])/len(comp_data_ip1['Concentration..ppb.']))**2))

In [69]:
# Yikes!
NSE1

-4.060104162103298e+55

In [5]:
# Now do this as a loop for simulations 1-5, and save their 5 NSEs
NSEs = []

for i in range(1,6):
    
    sim1_of26_data = pd.read_csv(output_path("outfall_31_26", i), usecols = [1], skiprows=5, names = ["davg_bif_conc"])*1000000
    sim1_of26_data['Sample_date'] = pd.date_range(start='1/1/2009', periods=3287, freq='D')
    sim1_of26_data['Site_code'] = '31_26'
    
    comp_data_ipi = sim1_of26_data.merge(obs_data, how = "inner", on = ['Sample_date','Site_code'])
    
    for of in outfalls[1:]:
        dat = pd.read_csv(output_path(of, i), usecols = [1], skiprows=5, names = ["davg_bif_conc"])*1000000
        dat['Sample_date'] = pd.date_range(start='1/1/2009', periods=3287, freq='D')
        dat['Site_code'] = of[-5:]
        comp_data_ipi = comp_data_ipi.append(dat.merge(obs_data, how = "inner", on = ['Sample_date','Site_code']), ignore_index=True)
        
    NSEi = 1 - ((sum(comp_data_ipi['davg_bif_conc'] - comp_data_ipi['Concentration..ppb.'])**2)/
                (sum(comp_data_ipi['Concentration..ppb.'] - sum(comp_data_ipi['Concentration..ppb.'])/
                     len(comp_data_ipi['Concentration..ppb.']))**2))
    NSEs.append(NSEi)

In [9]:
NSEs
# Wow, these are all terrrrible

[-4.060104162103298e+55,
 -3.420612276597484e+54,
 -7.583655349941465e+54,
 -3.366311251800084e+55,
 -1.329270805818393e+55]

In [39]:
# get details of top 25% of Nash-Sutcliffe Efficiencies, in this case let's just round up to 2

# borrowed from <https://stackoverflow.com/questions/13070461/get-indices-of-the-top-n-values-of-a-list>
top25pct = sorted(range(len(NSEs)), key=lambda i: NSEs[i])[-2:]

# get the lhs_parameters data
lhs_path = os.path.join(dir_path, "io", "lhs_sampled_params.csv")
lhsp_swmm = pd.read_csv(lhs_path, index_col=0)
# subset to just those pertaining to the winners
subset_lhs = lhsp_swmm.iloc[top25pct]
# make a new df for the ranges 
# start by importing the old one
old_param_ranges = pd.read_csv(os.path.join(dir_path, "input", "lhs", "lhs_param_ranges.csv"), index_col=0)
new_lhs_ranges = old_param_ranges
for idx, row in new_lhs_ranges.iterrows():
    new_lhs_ranges.loc[idx,'Min'] = min(subset_lhs[idx])
    new_lhs_ranges.loc[idx,'Max'] = max(subset_lhs[idx])
    new_lhs_ranges.loc[idx,'Range'] = max(subset_lhs[idx]) - min(subset_lhs[idx])
# save this as into a file in the folder with the old one
lhs_df.to_csv(os.path.join(dir_path, "input", "lhs", "lhs_param_ranges_subset.csv"))

In [41]:
new_lhs_ranges

,Min,Max,Range
Parameter,,,
NImperv,0.011321,0.021269,0.009948
NPerv,0.074908,0.390241,0.315334
SImperv,1.493654,1.628902,0.135248
SPerv,2.712322,4.380264,1.667942
PctZero,65.233530,90.839670,25.606141
MaxRate,56.813560,114.040617,57.227057
MinRate,4.666842,7.070918,2.404076
Decay,4.155502,5.120487,0.964985
DryTime,5.192462,6.291736,1.099275
